In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from urllib import request as rq
import urllib
import re
import math
import os
import csv
import time
import json
import pickle
import sys
import requests
from sklearn.cross_decomposition import PLSRegression
import matplotlib.pyplot as plt
cwd = os.getcwd()
print(cwd)

/home/liam/Documents/info370


In [3]:
def save_obj(obj, name ):
    with open(''+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('' + name + '.pkl', 'rb') as f:
        return pickle.load(f)
sys.setrecursionlimit(50000)

In [4]:
projectData = load_obj("projectData")
imdbIds = projectData["imdbIDs"]

In [5]:
# using imdb links gathered from the constructOscars Notebook
# collects data from imdb

imdbData = {}
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent} 
failedReqs = []


# gsearch = "http://www.imbd.com/title/"
for key, imdbID in imdbIds.items():
    link = "http://www.imbd.com/title/"+imdbID+"/"
    print(key)
    print(link)
    try:
        s = requests.Session()
        response = s.get(link, headers=headers).text
#         request=urllib.request.Request(link,None,headers) #The assembled request
#         response = urllib.request.urlopen(request)
        data = response # The data u need
        soup2 = bs(data, 'html.parser')
        
        try:
            mainSection = soup2.find("div",{"id":"main_top"})
            print(soup2)
        #     tbar = mainSection.find("div", class_="title_bar_wapper")
        #     ratingbar = tbar.find("div",class_="ratings_wrapper").find("div",class_="imdbRating")
        #     rating = ratingbar.find("div",class_="ratingValue").find("strong").find("span").contents[0]
            rating = mainSection.find("span",{"itemprop": "ratingValue"}).contents[0]

            votes = mainSection.find("span",{"itemprop": "ratingCount"}).contents[0]
            year = mainSection.find("span", {"id":"titleYear"}).find("a").contents[0]
            reviews = mainSection.find("div",class_="imdbRating").find("div", class_="hiddenImportant").find_all("span")
            userRevs = 0
            criticRevs = 0
            try:
                userRevs = reviews[0].contents[0].split()[0]
                criticRevs = reviews[1].contents[0].split()[0]
            except IndexError:
                print("couldnt't find the review tag")
                print(reviews)
                failedReqs.append(link)

            actors = []
            actortags = mainSection.find("div", class_="plot_summary").find_all("span", {"itemprop": "actors"})
            for tag in actortags:
                actors.append(tag.find("a").find("span").contents[0])

            details = soup2.find("div",{"id":"titleDetails"})
            runtime = details.find("time",{"itemprop":"duration"}).contents[0].split()[0]
            releasedate = soup2.find("meta",{"itemprop":"datePublished"})['content']
    #         print(key)
    #         print(link)
            print(rating)
            print(votes)
            print(userRevs)
            print(criticRevs)
            print(actors)
            print(runtime)
            imdbData[key] = [key, year, link, rating, votes, userRevs, criticRevs, actors, runtime, releasedate]
        except AttributeError as e:
            print("couldnt't find some attribute: %s" % e)
            print(link)
            failedReqs.append(link)
    except urllib.error.URLError as e:
        print(e.reason)  
        print(link)

        failedReqs.append(link)
    
    time.sleep(3)
print(failedReqs)
#     break
projectData['imdbData'] = imdbData
projectData['failedReqs'] = failedReqs
save_obj(projectData, "projectData")

The Song of Bernadette
http://www.imbd.com/title/tt0036377/


ConnectionError: HTTPConnectionPool(host='www.imbd.com', port=80): Max retries exceeded with url: /title/tt0036377/ (Caused by NewConnectionError('<requests.packages.urllib3.connection.HTTPConnection object at 0x7fb1a07014a8>: Failed to establish a new connection: [Errno -3] Temporary failure in name resolution',))

In [35]:
# ratings.txt is a data dump provided by imdb
# the following sections are attempting to parse it's poor format for our movie's ratings.
with open("ratings.txt","r") as fin:
    file=[line.replace("\n","").split(maxsplit=3) for line in fin]
print(file[0])
cols = file.pop(0)

['Distribution', 'Votes', 'Rank', 'Title']


In [37]:
# ratingsArr = np.array(file)

oscars = projectData['oscars']
df = pd.DataFrame(data=file, columns=cols)


['1927', '1st', 'False', 'The Racket', 'The Caddo Company']
['1928', '2nd', 'False', 'Alibi', 'Feature Productions']
['1929', '3rd', 'True', 'All Quiet on the Western Front', 'Universal']
['1930', '4th', 'True', 'Cimarron', 'RKO Radio']
['1931', '5th', 'False', 'Arrowsmith', 'Samuel Goldwyn Productions']
['1932', '6th', 'True', 'Cavalcade', 'Fox']
['1934', '7th', 'False', 'The Barretts of Wimpole Street', 'Metro-Goldwyn-Mayer']
['1935', '8th', 'False', 'Alice Adams', 'RKO Radio']
['1936', '9th', 'False', 'Anthony Adverse', 'Warner Bros.']
['1937', '10th', 'False', 'The Awful Truth', 'Columbia']
['1938', '11th', 'False', 'The Adventures of Robin Hood', 'Warner Bros.-First National']
['1939', '12th', 'False', 'Dark Victory', 'Warner Bros.-First National']
['1940', '13th', 'False', 'All This, and Heaven Too', 'Warner Bros.']
['1941', '14th', 'False', 'Blossoms in the Dust', 'Metro-Goldwyn-Mayer']
['1942', '15th', 'False', 'The Invaders', 'Ortus']
['1943', '16th', 'True', 'Casablanca', 'Wa

In [104]:
unfound = []
for o in oscars:
    for m in o:
        myear = m[0]
        mname = m[3]
        
#         print(myear)
        tnames = df[df['Title'].str.contains(mname)]
        tyear = tnames[tnames['Title'].str.contains(myear)]
        
        if tyear.size == 0:
            myear = str(int(myear)+1)
            tnames = df[df['Title'].str.contains(mname)]
            tyear = tnames[tnames['Title'].str.contains(myear)]
            if tyear.size == 0:
                print("failed to find one: %s" % mname)
                unfound.append(m)
            elif tyear.size > 1:
                maxVotes = tyear["Votes"].idxmax(1)
                tyear = tyear.loc[int(maxVotes)]
                rank = tyear["Rank"]
                votes = tyear["Votes"]
                imdbData[mname] = [mname, myear, rank, votes]
            else:
                print(tyear)
                rank = tyear.iloc[0]["Rank"]
                votes = tyear.iloc[0]["Votes"]
                imdbData[mname] = [mname, myear, rank, votes]
        elif tyear.size > 1:
            maxVotes = tyear["Votes"].idxmax(1)
            tyear = tyear.loc[int(maxVotes)]
            rank = tyear["Rank"]
            votes = tyear["Votes"]
            imdbData[mname] = [mname, myear, rank, votes]
        else:
            print(tyear)
            rank = tyear.iloc[0]["Rank"]
            votes = tyear.iloc[0]["Votes"]
            imdbData[mname] = [mname, myear, rank, votes]


failed to find one: The Patriot
failed to find one: Les Miserables
failed to find one: A Tale of Two Cities
failed to find one: Grand Illusion
failed to find one: The Invaders
failed to find one: Casablanca
failed to find one: In Which We Serve
failed to find one: Henry V
failed to find one: Great Expectations
failed to find one: All about Eve
failed to find one: Decision before Dawn
failed to find one: Around the World in 80 Days
failed to find one: Meredith Willson's The Music Man
failed to find one: How the West Was Won
failed to find one: Zorba the Greek
failed to find one: The Russians Are Coming The Russians Are Coming
failed to find one: The Emigrants
failed to find one: Cries and Whispers
failed to find one: The Godfather Part II
failed to find one: One Flew over the Cuckoo's Nest
failed to find one: Atlantic City
failed to find one: E.T. The Extra-Terrestrial
failed to find one: A Room with a View
failed to find one: Dances With Wolves
failed to find one: The Godfather, Part I

/home/liam/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
/home/liam/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:13: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


failed to find one: The Postman (Il Postino)
failed to find one: Life Is Beautiful
failed to find one: Crouching Tiger, Hidden Dragon
failed to find one: Precious: Based on the Novel 'Push' by Sapphire
failed to find one: Birdman or (The Unexpected Virtue of Ignorance)


In [106]:
# for those not found in the data dump look in omdb

genreq = "http://www.omdbapi.com/?tomatoes=true&t="
unfound2 = []
for m in unfound:
        e = False
        mname = str(m[3])
        myear = str(m[0]) 
        moviepage = rq.urlopen(genreq+urllib.parse.quote(mname) + "&y="+ urllib.parse.quote(myear))
        soup2 = bs(moviepage, 'html.parser')
        moviedat = json.JSONDecoder().decode(s=str(soup2))
        if "Error" in moviedat:
            e = True
            myear = str(int(myear) + 1) 
            moviepage = rq.urlopen(genreq+urllib.parse.quote(mname) + "&y="+ urllib.parse.quote(myear))
            soup2 = bs(moviepage, 'html.parser')
            moviedat = json.JSONDecoder().decode(s=str(soup2))
#             print(moviedat)
            if "Error" in moviedat:
                e = True
                unfound2.append(m)
        if not e:
            print(moviedat)
            imdbRating = moviedat['imdbRating']
            imdbVotes = ''.join(moviedat['imdbVotes'].split(","))
            imdbId = moviedat['imdbID']
#                 tomURevs = float(''.join(moviedat['tomatoUserReviews'].split(",")))
#                 tomURating = float(moviedat['tomatoUserRating'])
#                 tomUMeter = float(moviedat['tomatoUserMeter'])
#                 tomRating = float(moviedat['tomatoRating'])
#             runtime = float(moviedat['Runtime'].split()[0])
#             year = float(myear)

#             if m[2] == "True":
#                 win = 1
#             else:
#                 win = 0
            if imdbRating != "N/A" and imdbVotes != "N/A" and imdbId != "N/A":
                imdbData[mname] = [mname, myear, imdbRating, imdbVotes]
            else:
                unfound2.append(m)
print(unfound2)

{'Year': '1928', 'DVD': 'N/A', 'Country': 'USA', 'Awards': 'Won 1 Oscar. Another 4 nominations.', 'tomatoConsensus': 'N/A', 'tomatoImage': 'N/A', 'tomatoUserRating': '4.2', 'tomatoRotten': 'N/A', 'Director': 'Ernst Lubitsch', 'Language': 'English', 'Website': 'N/A', 'BoxOffice': 'N/A', 'Plot': 'In 18th-Century Russia, the Czar, Paul, is surrounded by murderous plots and trusts only Count Pahlen. Pahlen wishes to protect his friend, the mad king, but because of the horror of the ...', 'Response': 'True', 'tomatoUserReviews': '185', 'imdbID': 'tt0019257', 'imdbVotes': 'N/A', 'Title': 'The Patriot', 'tomatoRating': 'N/A', 'Writer': 'Ashley Dukes (play), Julian Johnson (titles), Hanns Kräly (adaptation), Dmitri Merezhkovsky (story "Paul I"), Alfred Neumann (play)', 'Actors': 'Emil Jannings, Florence Vidor, Lewis Stone, Vera Voronina', 'tomatoURL': 'http://www.rottentomatoes.com/m/the_patriot/', 'Metascore': 'N/A', 'Genre': 'Drama', 'Released': '01 Sep 1928', 'Runtime': '113 min', 'Rated': 

In [114]:
# assess those still missing
for l in unfound2:
    print('imdbData["'+l[3]+'"] = ["'+l[3]+'","'+l[0]+'", "", ""]')


imdbData["The Patriot"] = ["The Patriot","1928", "", ""]
imdbData["A Tale of Two Cities"] = ["A Tale of Two Cities","1936", "", ""]
imdbData["Grand Illusion"] = ["Grand Illusion","1938", "", ""]
imdbData["The Invaders"] = ["The Invaders","1942", "", ""]
imdbData["Casablanca"] = ["Casablanca","1943", "", ""]
imdbData["In Which We Serve"] = ["In Which We Serve","1943", "", ""]
imdbData["Henry V"] = ["Henry V","1946", "", ""]
imdbData["Great Expectations"] = ["Great Expectations","1947", "", ""]
imdbData["Around the World in 80 Days"] = ["Around the World in 80 Days","1956", "", ""]
imdbData["Meredith Willson's The Music Man"] = ["Meredith Willson's The Music Man","1962", "", ""]
imdbData["How the West Was Won"] = ["How the West Was Won","1963", "", ""]
imdbData["The Russians Are Coming The Russians Are Coming"] = ["The Russians Are Coming The Russians Are Coming","1966", "", ""]
imdbData["The Emigrants"] = ["The Emigrants","1972", "", ""]
imdbData["Cries and Whispers"] = ["Cries and Whis

In [115]:
# auto fill those still missing
imdbData["The Patriot"] = ["The Patriot","1928", "0", "0"]
imdbData["A Tale of Two Cities"] = ["A Tale of Two Cities","1936", "7.8", "3985"]
imdbData["Grand Illusion"] = ["Grand Illusion","1938", "8.1", "27154"]
imdbData["The Invaders"] = ["The Invaders","1942", "7.4", "4357"]
imdbData["Casablanca"] = ["Casablanca","1943", "8.6", "394552"]
imdbData["In Which We Serve"] = ["In Which We Serve","1943", "7.3", "3821"]
imdbData["Henry V"] = ["Henry V","1946", "7.3", "4519"]
imdbData["Great Expectations"] = ["Great Expectations","1947", "8.0", "17815"]
imdbData["Around the World in 80 Days"] = ["Around the World in 80 Days","1956", "6.8", "18320"]
imdbData["Meredith Willson's The Music Man"] = ["Meredith Willson's The Music Man","1962", "7.7", "13171"]
imdbData["How the West Was Won"] = ["How the West Was Won","1963", "7.1", "14447"]
imdbData["The Russians Are Coming The Russians Are Coming"] = ["The Russians Are Coming The Russians Are Coming","1966", "7.1", "6300"]
imdbData["The Emigrants"] = ["The Emigrants","1972", "8.0", "3577"]
imdbData["Cries and Whispers"] = ["Cries and Whispers","1973", "8.2", "20307"]
imdbData["Atlantic City"] = ["Atlantic City","1981", "7.4", "11587"]
imdbData["A Room with a View"] = ["A Room with a View","1986", "7.4", "30098"]
imdbData["Good Fellas"] = ["Good Fellas","1990", "8.7", "743934"]
imdbData["The Postman (Il Postino)"] = ["The Postman (Il Postino)","1995", "7.8", "26597"]
imdbData["Life Is Beautiful"] = ["Life Is Beautiful","1998", "8.6", "431336"]
imdbData["Precious: Based on the Novel 'Push' by Sapphire"] = ["Precious: Based on the Novel 'Push' by Sapphire","2009", "7.3", "88297"]

In [117]:
print(len(imdbData))
projectData['imdbData'] = imdbData
save_obj(projectData, "projectData")

516


In [124]:
mvs = []
for o in oscars:
    for m in o:
        votes = imdbData[m[3]][3]
        rank = imdbData[m[3]][2]
        new = [m[0], m[1], m[2], m[3], m[4], rank, votes]
        mvs.append(new)
        
f = open("AA_bestpictures2.csv", 'w')
fields = "; ".join(["year", "Academy Award", "Winner", "Movie Name", "Producer", "IMDB Rating", "IMDB Votes"]) + "\n"
joined = "\n".join(["; ".join([i for i in m]) for m in mvs])
joined = fields + joined
f.write(joined)
f.close()